![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# PLAN YOUR TRIP WITH KAYAK
## _Data Collection and Management Project_

### Le Projet

L'équipe marketing de Kayak a découvert que 70 % de leurs utilisateurs qui planifient un voyage aimeraient avoir plus d'informations sur la destination vers laquelle ils se rendent.
Par conséquent, l'équipe souhaite créer une application qui recommandera aux gens où planifier leurs prochaines vacances en se basant sur deux variables :
* Météo
* Hôtels dans la région

Le projet débutant à peine, mon travail consiste à :
* Récupérer les coordonnées GPS des destinations
* Obtenir des données météorologiques pour chaque destination
* Obtenir des informations sur les hôtels pour chaque destination
* Stocker toutes les informations ci-dessus dans un _data lake_
* Extraire, transformer et charger les données nettoyées du _data lake_ vers un _data warehouse_

Avant de commencer, j'ai créé une liste de tâches à faire et je les ai triées par ordre de priorité. J'ai aussi estimé combien de temps chaque tâche me prendrait. Enfin, j'ai mis en place un projet GitHub sous forme de Kanban afin de suivre l'état d'avancée des tâches.

!["project"](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/project.png)

### 1. Récupérer les données météo

Grâce à `nominatim.streetmap.org`, je peux obtenir les coordonnées GPS d'une sélection de villes.

In [ ]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv("./.env"))

Ci-dessous la liste des villes que l'équipe de Kayak veut analyser pour commencer.

In [ ]:
CITIES =[
    "Mont Saint Michel",
    "St Malo",
    "Bayeux",
    "Le Havre",
    "Rouen",
    "Paris",
    "Amiens",
    "Lille",
    "Strasbourg",
    "Chateau du Haut Koenigsbourg",
    "Colmar",
    "Eguisheim",
    "Besancon",
    "Dijon",
    "Annecy",
    "Grenoble",
    "Lyon",
    "Gorges du Verdon",
    "Bormes les Mimosas",
    "Cassis",
    "Marseille",
    "Aix en Provence",
    "Avignon",
    "Uzes",
    "Nimes",
    "Aigues Mortes",
    "Saintes Maries de la mer",
    "Collioure",
    "Carcassonne",
    "Ariege",
    "Toulouse",
    "Montauban",
    "Biarritz",
    "Bayonne",
    "La Rochelle"
]

In [5]:
def get_cities_coordinates(cities: list[str]) -> list[dict]:
    url = "https://nominatim.openstreetmap.org"
    endpoint = "search"
    data = []
    for city in cities:
        city = city.lower()
        payload = {"city": city, "format": "json"}
        response = requests.get(f"{url}/{endpoint}", params=payload)

        if response.status_code != 200:
            print(f"Failed with '{city}' -> {response.status_code}")
            continue

        resp = response.json()[0]
        data.append({"city": city,
                        "lat": resp.get("lat", None),
                        "lon": resp.get("lon", None)})

    return data

Toujours via requests, je récupère des données sur la météo à venir dans une sélection de villes.

Pour cela, j'utilise l'API openweathermap.org. La stratégie reste la même si ce n'est que cette API a besoin d'une API KEY pour exécuter des requêtes.

Je décide de récupérer les données météorologiques des 7 jours à venir.

Le but est de déterminer dans quelles villes le temps sera le plus agréable.

Pour cela, je me base sur ces éléments :

* la température minimum (`temp_min`)
* la température maximum (`temp_max`)
* le pourcentage d'humidité (`humidity`)
* le pourcentage de couverture nuageuse (`clouds`)
* la probabilité qu'il pleuve (`rain_prob`)

In [6]:
def get_weather_data(cities_coordinates: list[dict]) -> list[dict]:
    url = "https://api.openweathermap.org/data/2.5"
    endpoint = "forecast"
    data = []
    city_id = 1
    for row in cities_coordinates.copy():
        payload = {"lat": float(row["lat"]),
                    "lon": float(row["lon"]),
                    "units": "metric",
                    "cnt": 7,  # 7 days to come
                    "appid": os.getenv("WEATHER_KEY")}

        response = requests.get(f"{url}/{endpoint}",
                                params=payload)

        if response.status_code != 200:
            print(f"Failed -> {response.status_code}")
            continue

        day_id = 1
        for dt in response.json()["list"]:
            data.append({
                "city_id": city_id,
                "city": row["city"],
                "lat": float(row["lat"]),
                "lon": float(row["lon"]),
                "day_id": day_id,
                "temp": dt["main"].get("temp", None),
                "temp_min": dt["main"].get("temp_min", None),
                "temp_max": dt["main"].get("temp_max", None),
                "humidity": dt["main"].get("humidity", None),
                "clouds": dt["clouds"].get("all", None),
                "rain_prob": dt.get("pop", None),
            })
            day_id += 1
        city_id += 1

    return data

Maintenant que les fonctions sont écrites, il n'y a plus qu'à enregistrer le résultat dans un fichier CSV !
_(Pour des raisons de gains de place, les conventions PEP8 ne sont pas respectées dans la cellule ci-dessous)._

In [ ]:
df = pd.DataFrame(get_weather_data(get_cities_coordinates(CITIES)))
df.to_csv("weather_data.csv", index=False, encoding="utf-8")

### 2. Récupérer les données sur les hôtels

Le client veut se baser sur le site _booking.com_ pour collecter ses données. Comme il n'y a pas d'API officielle, il faut faire du web scraping pour obtenir les informations nécessaires.

Je simule donc une recherche sur le site et récupère tous les hôtels proposés grâce au CSS. Après avoir stocké leur nom, note et adresse URL, je simule un clic sur les hôtels pour accéder à leurs détails (voir méthode `self.parse_hotel_detail`).


In [ ]:
import scrapy


class BookingSpider(scrapy.Spider):
    name = "booking"
    allowed_domains = ["booking.com"]
    cities = ["montpellier"]
    start_urls = [f"https://www.booking.com/searchresults.html?ss={city}" for city in CITIES]

    custom_settings = {
        "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "ROBOTSTXT_OBEY": False
    }

    def parse(self, response):
        hotels = response.css('div[data-testid="property-card"]')

        for hotel in hotels:
            city = response.url.split("=")[-1]
            name = hotel.css('div[data-testid="title"]::text').get().strip()
            rating = hotel.css("div.f13857cc8c.e008572b71::text").get()
            if not rating:
                rating = "0.0"
            url = hotel.css('a[data-testid="title-link"]::attr(href)').get()

            yield response.follow(url, callback=self.parse_hotel_detail,
                                  meta={"name": name,
                                        "rating": rating,
                                        "city": city
                                        })

        next_page = response.css("a.bui-pagination__link.pagenext::attr(href)").get()
        if next_page:
            yield response.follow(next_page, self.parse)

    def parse_hotel_detail(self, response):
        city = response.meta["city"]
        name = response.meta["name"]
        url = response.url
        rating = response.meta["rating"]
        address = response.css("span.hp_address_subtitle.js-hp_address_subtitle.jq_tooltip::text").get().strip()
        description = response.css('p[data-testid="property-description"]::text').get().strip()
        coordinates = response.css("a#hotel_address::attr(data-atlas-latlng)").get()

        if coordinates:
            latitude, longitude = coordinates.split(",")
            latitude = float(latitude)
            longitude = float(longitude)
        else:
            latitude = None
            longitude = None

        yield {
            "city": city,
            "name": name,
            "url": url,
            "rating": rating,
            "address": address,
            "description": description,
            "latitude": latitude,
            "longitude": longitude
        }

Les données ont été enregistrées dans un fichier JSON, il faut donc les convertir en CSV.

In [ ]:
with open("hotels.json", "r", encoding="utf-8") as file:
    hotel_content = file.read()

df = pd.DataFrame(eval(hotel_content))
df.to_csv("csv_files/weather_data.csv", index=False, encoding="utf-8")

### 3. Charger les données dans un datalake

Il y a maintenant 2 fichiers CSV contenant des informations sur la météo et sur les hôtels des villes.

Avec boto3, je crée un bucket sur AWS S3 afin d'y charger les fichiers.

In [ ]:
import os
import boto3

BUCKET_NAME = "tgiandoriggio-bucket-kayak-01"

root = "csv_files"
filenames = ["weather_data.csv",
             "hotel_data.csv"]

session = boto3.Session(aws_access_key_id=os.getenv("AWS_KEY"),
                        aws_secret_access_key=os.getenv("AWS_SECRET"))

s3 = session.resource("s3")
bucket = s3.create_bucket(Bucket=BUCKET_NAME)

for file in filenames:
    path = f"{root}/{file}"
    if not os.path.isfile(path):
        raise FileExistsError(f"{path} does not exist.")

    data = pd.read_csv(path)
    csv = data.to_csv(index=False)
    put_object = bucket.put_object(Key=file, Body=csv)

![s3_bucket](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/s3_bucket.png)

### 4. Télécharger les données depuis le datalake

Les fichiers ont été chargés, j'aimerais maintenant les télécharger. Le but ici est avant tout de manipuler les espaces de stockage afin d'anticiper d'éventuels problèmes à venir.

Dans cette fonction, j'initialise une nouvelle session pour récupérer le bucket qui m'intéresse. De ce bucket, je télécharge les fichiers CSV et les stocke dans des fichiers temporaires dans le but de les convertir en Dataframe Pandas.

In [ ]:
BUCKET_NAME = "tgiandoriggio-bucket-kayak-01"
KEYS = ["weather_data.csv", "hotel_data.csv"]


def get_dataframes_from_s3() -> list[pd.DataFrame, pd.DataFrame]:
    # init session and bucket
    session = boto3.Session(aws_access_key_id=os.getenv("AWS_KEY"),
                            aws_secret_access_key=os.getenv("AWS_SECRET"))

    s3 = session.resource("s3")
    bucket = s3.Bucket(BUCKET_NAME)

    dataframes = []
    for obj in bucket.objects.all():
        file_key = obj.key
        file_path = "AppData/Local/Temp/" + file_key

        try:
            bucket.download_file(file_key, file_path)
            dataframes.append(pd.read_csv(file_path))

        except Exception as e:
            print(f"{file_key} -> {e}")

    return dataframes


hotel_df, weather_df = get_dataframes_from_s3()

print(hotel_df.head())
print(weather_df.head())

### 5. Envoyer les données dans un data warehouse

L'équipe de Kayak veut accéder aux données via des requêtes SQL.

Pour satisfaire cette requête, je crée une base de données et y charge nos données. J'opte pour la solution AWS RDS car elle permet de créer des bases de données relationnelles très rapidement. De plus, celles-ci sont facilement scalables.
L'engine préféré sera `PostgreSQL`.

!["rds](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/rds_database.png)

J'utilise `sqlalchemy` pour gérer la base de données avec Python.

In [ ]:
from pathlib import Path
from sqlalchemy import create_engine, text
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

La base de données étant créée, il faut maintenant s'y connecter.

In [ ]:
database_url = os.getenv("RDS_ENDPOINT")
if not database_url:
    raise ValueError("Please set the RDS_ENDPOINT environment variable.")

engine = create_engine(database_url, echo=True)
Base = declarative_base()
conn = engine.connect()

Pour créer la structure de la base de données, je définis des tables via des classes.

La classe `City` contient l'ID, le nom, et les coordonnées GPS des villes.

La classe `Weather` contient l'ID du jour de prévision, la température actuelle / minimale / maximale, les pourcentages d'humidité et de couverture nuageuse ainsi que la probabilité qu'il pleuve.

Il est important de noter que ces deux classes sont liées entre elles par des jointures. Le but est d'optimiser le nombre de colonnes et de valeurs dupliquées.
Les données météo donnent des infos sur les 7 jours à venir pour chaque ville. Si je mets les données météo et les données GPS dans la même table, alors il y aurait 7 fois les mêmes données GPS.

In [ ]:
class City(Base):
    __tablename__ = "cities"
    id = Column(Integer, primary_key=True)
    city = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    weather = relationship("Weather", back_populates="city_rel")
    hotels = relationship("Hotel", back_populates="city_rel")


class Weather(Base):
    __tablename__ = "weather"
    id = Column(Integer, primary_key=True)
    city_id = Column(Integer, ForeignKey("cities.id"))
    day_id = Column(Integer)
    temp = Column(Float)
    temp_min = Column(Float)
    temp_max = Column(Float)
    humidity = Column(Integer)
    clouds = Column(Integer)
    rain_prob = Column(Float)
    city_rel = relationship("City", back_populates="weather")


class Hotel(Base):
    __tablename__ = "hotels"
    id = Column(Integer, primary_key=True)
    city_id = Column(Integer, ForeignKey("cities.id"))
    name = Column(String)
    url = Column(String)
    rating = Column(Float)
    address = Column(String)
    description = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    city_rel = relationship("City", back_populates="hotels")


Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

Passons maintenant à l'ajout de nos données dans les tables. Pour cela, je récupère les fichiers CSV téléchargés depuis S3 et les convertis en dataframes.

Pour des raisons de lisibilité, je crée une petite fonction qui nettoie les noms des hôtels. Si je choisis de le faire à cette étape et non plus tôt, c'est pour illustrer le fait que les données du datalake, en plus d'être éparpillées, ne sont pas toujours propres. Un dernier check est donc le bienvenu.

In [ ]:
def read_csv(file: Path) -> pd.DataFrame:
    if not file.is_file():
        raise FileExistsError()
    return pd.read_csv(file)


def clean_city_name(city_name: str) -> str:
    return city_name.lower().replace(" ", "").replace("%20", "")


weather_df = read_csv(root / "weather_data.csv")
hotel_df = read_csv(root / "hotel_data.csv")

Ajoutons les données !

Pour cela, j'opte pour une itération sur toutes les lignes des dataframes qui envoie petit à petit les valeurs dans les tables. Il y a des moyens plus rapides de le faire, mais par souci de gestion du temps, je privilégie le code fonctionnel à un code optimisé.

In [ ]:
# Insert City data
weather_df["city"] = weather_df["city"].apply(clean_city_name)
cities = weather_df[
    ["city_id", "city", "lat", "lon"]
    ].drop_duplicates().to_dict(orient="records")

for city in cities:
    city_record = City(id=city["city_id"],
                       city=city["city"],
                       latitude=city["lat"],
                       longitude=city["lon"])
    session.add(city_record)

# Insert Weather data
for i, row in weather_df.iterrows():
    weather = Weather(id=i,
                      city_id=row["city_id"],
                      day_id=row["day_id"],
                      temp=row["temp"],
                      temp_min=row["temp_min"],
                      temp_max=row["temp_max"],
                      humidity=row["humidity"],
                      clouds=row["clouds"],
                      rain_prob=row["rain_prob"])
    session.add(weather)

# Insert Hotel data
hotel_df["city"] = hotel_df["city"].apply(clean_city_name)

for i, row in hotel_df.iterrows():
    city = session.query(City).filter_by(city=row["city"]).first()
    try:
        hotel = Hotel(id=i,
                      city_id=city.id,
                      name=row["name"],
                      url=row["url"],
                      rating=row["rating"],
                      address=row["address"],
                      description=row["description"],
                      latitude=row["latitude"],
                      longitude=row["longitude"])
        session.add(hotel)
    except Exception as e:
        print(e)
        continue

session.commit()

Ci-dessous un exemple de requête pour vérifier si nos données sont bien importées dans les tables. Je pourrais écrire dans Visual Studio Code `conn.execute(text("SELECT * FROM cities))`.

Le but ici est de prendre en main Azure Data Studio.

!["cities](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/select_cities.png)
!["weather](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/select_weather.png)
!["hotel](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/select_hotels.png)

### 6. Visualisation

Enfin, parce que des valeurs dans un tableau ne sont pas très parlantes, faisons un peu de visualisation !

Je vais utiliser `plotly`.

In [ ]:
import plotly.graph_objects as go

Je me connecte à la base de données et crée des dataframes pour exploiter son contenu.

In [ ]:
# Init database connection
engine = create_engine(os.getenv("RDS_ENDPOINT"), echo=True)
conn = engine.connect()

# Init pandas dataframes
weather_df = pd.read_sql("SELECT * FROM weather", conn)
cities_df = pd.read_sql("SELECT * FROM cities", conn)
hotels_df = pd.read_sql("SELECT * FROM hotels", conn)

conn.close()

Je crée 2 cartes :

* La première montre les 10 meilleures villes où partir en vacances au cours des 7 prochains jours selon la météo.
* La seconde indique où trouver les 20 meilleurs hôtels dans chaque ville.

##### Les meilleures destinations de vacances

Tout d'abord, je fusionne deux dataframes (`weather` et `city`).

`pd.merge()` crée deux colonnes dupliquées. Je les supprime avec `.drop`.

Ensuite, je groupe chaque ville en fonction de plusieurs fonctions d'agrégation. Je récupère la moyenne pour la température maximale, l'humidité, la couverture nuageuse et la probabilité de pluie. Cela me permet d'avoir une estimation de la météo moyenne au cours des 7 prochains jours.

Enfin, je trie les villes par température maximale la plus élevée et autres paramètres au plus bas.

Le graphique est construit avec une `Scattermapbox` dont la taille et couleur des points est déterminée en fonction de la température maximale moyenne.

In [ ]:
def get_best_cities() -> pd.DataFrame:
    # Merge dataframes : weather_df["city_id"] -> cities_df["id"]
    merged_df = pd.merge(weather_df, cities_df,
                         left_on="city_id", right_on="id",
                         suffixes=("_weather", "_city"))

    merged_df = merged_df.drop(columns=["id_city", "id_weather"])  # duplicates

    avg_df = merged_df.groupby(["city_id", "city", "latitude", "longitude"]) \
        .agg({"temp_max": "mean",
              "humidity": "mean",
              "clouds": "mean",
              "rain_prob": "mean"}) \
        .reset_index()

    avg_df = avg_df.sort_values(
        by=["temp_max", "humidity", "clouds", "rain_prob"],
        ascending=[False, True, True, True]).head(10)

    return avg_df


def create_best_cities_figure(df: pd.DataFrame) -> go.Figure:
    cities_map = go.Figure(go.Scattermapbox(
        lat=df["latitude"],
        lon=df["longitude"],
        text=df["city"],
        mode="markers",
        marker=dict(
            size=df["temp_max"],
            color=df["temp_max"],
            colorscale="bluered",
            showscale=True,
            colorbar=dict(title="Max Temperature"),
            sizemode="area",
            sizeref=2.*max(df['temp_max'])/(35.**2),
            sizemin=4
        )
    ))

    cities_map.update_layout(
        mapbox=dict(
            style='open-street-map',
            zoom=5,
            center=dict(lat=df['latitude'].mean(), lon=df['longitude'].mean())
        ),
        title={
            "text": "<b>The best destinations for the next 7 days according to the weather forecast</b>",
            "x": 0.5
        },
        margin={"r": 0, "t": 50, "l": 0, "b": 0}
    )

    return cities_map

##### Les meilleurs hôtels

La procédure est beaucoup plus simple. Je trie simplement les hôtels en fonction de leur note et récupère les 20 premiers.

Le graphique est, lui, bien différent. Si j'utilise toujours une `Scattermapbox`, j'ai décidé de créer un menu déroulant pour sélectionner les villes dans le but de faciliter l'expérience utilisateur.

Dans un premier temps, je cherche à empiler toutes les cartes les unes sur les autres mais ce n'était pas du tout une solution adéquate.

Je crée donc une seule instance et mets à jour le centre de la carte à chaque sélection de ville.

Pour placer les villes au centre de la carte, je fais la moyenne des coordonnées GPS des hôtels.

In [ ]:
def get_best_hotels() -> pd.DataFrame:
    df = hotels_df.groupby("city_id", as_index=False) \
        .apply(
            lambda x: x.sort_values(
                by="rating", ascending=False).head(20)).reset_index()

    return df.drop(columns=["level_0", "level_1"])


def create_best_hotels_figure(df: pd.DataFrame) -> go.Figure:
    coords_df = df.groupby("city_id") \
                   .agg({"latitude": "mean", "longitude": "mean"}) \
                   .reset_index()
    # need to attribute the city name to its id again
    coords_df = coords_df.merge(cities_df[['id', 'city']],
                                left_on='city_id', right_on='id',
                                how='left')
    coords_df.drop('id', axis=1, inplace=True)

    fig = go.Figure()

    fig.add_trace(
        go.Scattermapbox(
            lat=df["latitude"],
            lon=df["longitude"],
            text='<b>' + df["name"] + "</b><br>" + df["rating"].astype(str) + "/10",
            mode="markers",
            marker=dict(
                size=df["rating"],
                color=df["rating"],
                colorscale="hot",
                showscale=True,
                sizemode="area",
                sizeref=2.*max(df['rating'])/(35.**2),
                sizemin=1
            )
        )
    )

    fig.update_layout(
        title=go.layout.Title(text="<b>Top 20 best hotels</b>", x=0.5),
        showlegend=False,
        mapbox=dict(
            style="open-street-map",
            zoom=5,
            center=dict(lat=df["latitude"].mean(), lon=df["longitude"].mean())
        )
    )

    buttons = [
        go.layout.updatemenu.Button(
            label=str(r["city"]),
            method="update",
            args=[
                {"visible": True},
                {"mapbox": {
                    "style": "open-street-map",
                    "zoom": 12,
                    "center": {"lat": r["latitude"], "lon": r["longitude"]}}}]
        ) for _, r in coords_df.iterrows()
    ]

    fig.update_layout(
        updatemenus=[go.layout.Updatemenu(buttons=buttons)]
    )

    return fig

Les fonctions étant écrites, il ne reste plus qu'à les appeler. Je pourrais ne pas créer de fonctions mais je fais ce choix pour des raisons de lisibilité du code.

In [ ]:
df_cities = get_best_cities()
fig_cities = create_best_cities_figure(df_cities)

df_hotels = get_best_hotels()
fig_hotels = create_best_hotels_figure(df_hotels)

fig_cities.show()
fig_hotels.show()

!["cities](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/fig_cities.png)
!["hotels](https://raw.githubusercontent.com/tristanGIANDO/jedha_bootcamp_projects/main/bloc_01_Data_Infrastructure/plan_your_trip_with_kayak/src/images/fig_hotels.png)

## Conclusion

Les données sont collectées au mois de juin 2024. La tendance des résultats oriente l'utilisateur à choisir une destination dans le Sud-Ouest de la France.

Les scripts doivent être relancés régulièrement. Il sera probablement intéressant pour un utilisateur de connaître les meilleures destinations du mois ou de l'année, en stockant les données sur de longues périodes.